# 01.112 Machine Learning Design Project

## About the Project

We have 4 datasets in the `/data` folder. For each dataset, there is: 
- a labelled training set train, 
- an unlabelled development set `dev.in`
- a labelled development set `dev.out` 

The labelled data has the format of: `token` `\t` `tag`
- one token per line
- token and tag separated by tab 
- single empty lines that separates sentences

For the labels, they are slightly different for different datasets.
- SG, CN (Entity):
    - B-*: Beginning of entity
    - I-*: Inside of entity
    - O: Outside of any entity
- EN, AL (Phrase):
    - B-VP: Beginning of Verb Phrase
    - I-VP: Inside of Verb Phrase
    - *-NP: Noun Phrase
    - *PP: Propositional Phrase
    - O: Outside of any phrase

*Goal*: Build sequence labelling systems from training data (x) and use it to predict tag sequences for new sentences (y).

## Team members 
- Andri Setiawan Susanto
- Eldon Lim 
- Tey Siew Wen

## Part 1
Already completed individually.

## Part 2

a) Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):

b)

1. Make a modified training set by replacing those words that appear $<k$ times in the training set with a special word token `#UNK#` before training.
2. During testing phase, ifaworddoesnot appear in the modified training set, we also replace that wordwith `#UNK#`.
3. Compute Emission Paramters with the function in (a)

For all the four datasets EN, AL, CN, and SG, learn these parameters with `train`, and evaluate your
system on the development set `dev.in` for each of the dataset. Write your output to `dev.p2.out`
for the four datasets respectively. Compare your outputs and the gold-standard outputs in `dev.out`
and report the precision, recall and F scores of such a baseline system for each dataset.

In [11]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

def emissionPara(arr, k, replaceWord):
    """
    calculates the emission probabilities given a numpy array containing arrays of [(x,y)]
    
    returns:
    emission_dict => a dictionary in the form of {(x,y): probability}
    valid_x => list of x that has occurences more or equal to k times.
    xy_dic => a dictionary in the form of {x: y} where y is the most probable label given x
    """
    x_counter = defaultdict(int)
    y_counter = defaultdict(int)
    xy_counter = defaultdict(int)
    x_labels = defaultdict(list)
    emission_params = {}
    xy_dict = {}

    obs = 0
    for x_y in arr:
        x, y = x_y[0].split(" ")
        obs += 1
        x_counter[x] += 1
        y_counter[y] += 1
        xy_counter[x,y] += 1
        if y not in x_labels[x]:
            x_labels[x].append(y)
        
    x_to_replace = [x for x in x_counter if x_counter[x] < k]
    print(f"There are {obs} observations, {len(x_to_replace)} values of x is to be replaced ")

    # removing x occurences less than k times from x_counter and its labels
    for r in x_to_replace:
        for label in x_labels[r]:
            if label not in x_labels[replaceWord]:
                x_labels[replaceWord].append(label)
            xy_counter[replaceWord, label] += xy_counter[r, label]
            
            del xy_counter[r,label]
        del x_counter[r]
        del x_labels[r]
        
    # calculation of emission probabilities        
    for x_y, x_y_count in xy_counter.items():
        y = x_y[1]
        emission_params[x_y] = x_y_count / y_counter[y]

    # get best labels
    for x, labels in x_labels.items():
        emission_probs = np.zeros(len(labels))
        for i in range(len(labels)):
            label = labels[i]
            emission_probs[i] = emission_params[x,label]
        xy_dict[x] = labels[np.argmax(emission_probs)]
    return emission_params, x_counter.keys(), xy_dict

def predict(data, xy_dict, replaceWord):
    print("Predicting labels")
    result = pd.DataFrame()
    
    def replace_string(x):
        if x not in xy_dict:
            return "{} {}".format(replaceWord, xy_dict[replaceWord])
        else:
            return "{} {}".format(x, xy_dict[x])
         
    return data["x"].apply(lambda s: replace_string(s) if str(s) != "nan" else " ")

## Part 3

Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation):

In [12]:
def split_into_columns(df_column):
    new = df_column.str.split(" ", n=1, expand=True)
    return new[0], new[1]

In [13]:
from collections import Counter, defaultdict

def transitionPara(data):
    y_count = defaultdict(int)
    subseq_count = defaultdict(int)
    
    for i in range(len(data) - 1):
        curr_xy = data[i][0]
        next_xy = data[i+1][0]
        
        if i == 0:
            x, y = curr_xy.split(" ")
            subseq_count["START", y] += 1
            y_count[y] += 1
            y_count["START"] += 1
            continue
        
        elif curr_xy == None or curr_xy == np.nan or str(curr_xy) == "nan":
            x, y = next_xy.split(" ")
            subseq_count["START", y] += 1
            y_count[y] +=1
            y_count["START"] += 1
        
        elif i+1 == len(data) or next_xy == None or next_xy == np.nan or str(next_xy) == "nan":
            x, y = curr_xy.split(" ")
            subseq_count[y,"END"] +=1
            y_count[y] += 1
            y_count["END"] += 1
            
        else:
            y1 = curr_xy.split()[1]
            y2 = next_xy.split()[1]
            subseq_count[y1,y2] +=1
            y_count[y1] += 1
        
        # Calculation of transition params
    transition_dict = {}

    for k,v in subseq_count.items():
        y1 = k[0]
        y2 = k[1]
        transition_dict[y1,y2] = subseq_count[y1,y2] / y_count[y1]
    
    del y_count["END"]
    del y_count["START"]

    return transition_dict, subseq_count, y_count

In [14]:
def viterbi(unique_word_list):
    #This is for the starting for viterbi
    global nodes
    
    num_nodes_per_col = len(nodes)
    store=np.zeros(num_nodes_per_col)   #store = the storage for scores for all the nodes. 
    scorelist=np.zeros((num_nodes_per_col, len(unique_word_list) + 1))
    rankings = np.zeros((num_nodes_per_col, len(unique_word_list) + 1))
    
    for i in range(num_nodes_per_col):
        emission_score = emission(nodes[i],unique_word_list[0])
        transition_score = transition("START",nodes[i])
        if transition_score == 0 or emission_score == 0:
            store[i] = -1e100
        else:
            store[i] = np.log(emission_score)+np.log(transition_score)
            
    scorelist[:,0] = store
    store = np.zeros(num_nodes_per_col)
    score_per_node=np.zeros(num_nodes_per_col)
    
    #This is for the middle portion for viterbi
    #score per node = prevnode*emission*transition

    if len(unique_word_list)>1:
        for j in range(len(unique_word_list)-1): #for the whole length in sentence
            for k in range(num_nodes_per_col): #for each node
                for l in range(num_nodes_per_col): #for 1 node, transition from prev node to current node
                    prev_node = scorelist[l][j]
                    emission_score = emission(nodes[k],unique_word_list[j+1])
                    transition_score = transition(nodes[l],nodes[k])
                    
                    if transition_score == 0 or emission_score == 0:
                        score_per_node[l] = np.NINF
                    else:   
                        score_per_node[l] = prev_node+np.log(emission_score)+np.log(transition_score) 
                
                store[k] = np.max(score_per_node) # max path
                score_per_node=np.zeros(num_nodes_per_col)
            
            scorelist[:,j+1] = store
            store = np.zeros(num_nodes_per_col)
                      
        score_at_stop=np.zeros(num_nodes_per_col)
        
        #This is for the STOP for viterbi
        for m in range(num_nodes_per_col):
            score_at_stop[m] = np.log(transition(nodes[m],"END")) + scorelist[m][len(unique_word_list)-1]
        scorelist[:,-1] = np.full(num_nodes_per_col,np.max(score_at_stop))
        
    return scorelist
  
def viterbi_backtrack(scorelist):
    """
    back tracking for viterbi
    node value*transition = array, then find max, then find position. use position for next step.
    np.argmax returns index of max in the element.
    The final score on the score list is for end
    """ 
    global nodes
    
    scorelist = np.flip(scorelist,axis=1) #reverse the score list so easier to calculate.
    
    max_node_index = 0 
    num_obs = scorelist.shape[1]
    num_nodes = scorelist.shape[0]
    node_holder = np.zeros(num_nodes)
    path = []

    if (num_obs == 1):
        for k in range (num_nodes):
            calculate_max_node = scorelist[0][k] + np.log(transition(nodes[k],"END"))
            node_holder[i] = calculate_max_node
        path.append(nodes[np.argmax(node_holder)])
        return(path[::-1])

    for i in range (1,num_obs): # for length of sentence
        for j in range(num_nodes): #for each node
            if (i==1):
                calculate_max_node = scorelist[j][i] + np.log(transition(nodes[j],"END"))
                node_holder[j] = calculate_max_node
            else:
                calculate_max_node = scorelist[j][i] + np.log(transition(nodes[j],nodes[max_node_index]))
                node_holder[j] = calculate_max_node
        
        max_node_index=np.argmax(node_holder)
        path.append(nodes[np.argmax(node_holder)])
        node_holder=np.zeros(num_nodes)

    return(path[::-1])

def emission(node,word):
    global emission_dict
    global nodes
    pair = word,node
    detector = 0 # this is used to find if word exist in the dictionary
    if pair not in emission_dict.keys(): #if the combination cannot be found in the dictionary
                                         #Either the word exists, or word is new. 
        for o in nodes:
            missing_pair = word,o
            if missing_pair in emission_dict.keys(): #
                detector = 1 # to detect if word exist in dictionary.
                break
        if detector == 1:
            score=0   #this means that this node is not the correct node.
        else:
            replaced_text = "#UNK#",node
            if replaced_text in emission_dict.keys():
                score = emission_dict[replaced_text] #if label have #unk#
                
            else:
                score = 0   #if label does not have #unk#, then set to 0.
    else:
        score = emission_dict[pair]
    return score


def transition(x1,x2):
    global transition_dic
    #will use this to search the transition from x1 to x2
    pair = x1,x2
    if pair not in transition_dic.keys():
        score = 0
    else:
        score = transition_dic[x1,x2]
    return score

In [15]:
import numpy as np
import heapq 

def kViterbiParallel(nodes, obs, num_k):
    global transition
    global emission
    
    """
    scores = highest probability of any path that reaches point i
    phi(i, k) = kth lowest cost to reach node i at col t from node 1 at col 0.
    rank = The ranking of multiple paths through a state
    """
    num_nodes = len(nodes)
    num_words = len(obs)

    scores = np.zeros((num_words+1, num_nodes, num_k),float)
    phi = np.zeros((num_words+1, num_nodes, num_k),int)
    rank = np.zeros((num_words+1, num_nodes, num_k),int)

    # for k in range(K):
    for i in range(num_nodes):
        curr_emission = emission(nodes[i], obs[0])
        curr_transition = transition("START", nodes[i])
        scores[0, i, 0] = np.log(curr_emission) + np.log(curr_transition)
        phi[0, i, 0] = i

        # Set the other options  to 0 initially
        for k in range(1, num_k):
            scores[0, i, k] = 0.0
            phi[0, i, k] = i

    # Go forward calculating top k scoring paths
    # for each state s1 from previous state s2 at time step t
    for t in range(1, num_words-1):
        for s1 in range(num_nodes):
          
            h = []  # heap structure for priority queue 
          
            for s2 in range(num_nodes):
                for k in range(num_k):
                    curr_emission = emission(nodes[s1], obs[t])
                    curr_transition = transition(nodes[s1], nodes[s2])
                    prob = scores[t - 1, s2, k] + np.log(curr_transition) + np.log(curr_emission)

                    state = s2

                    # Push the probability and state that led to it
                    heapq.heappush(h, (prob, s2))

            #Get the sorted list
            h_sorted = [heapq.heappop(h) for i in range(len(h))]
            h_sorted.reverse()

            #We need to keep a ranking if a path crosses a state more than once
            rankDict = dict()

            #Retain the top k scoring paths and their phi and rankings
            for k in range(0, num_k):
                scores[t, s1, k] = h_sorted[k][0]
                phi[t, s1, k] = h_sorted[k][1]

                state = h_sorted[k][1]

                if state in rankDict:
                    rankDict[state] = rankDict[state] + 1
                else:
                    rankDict[state] = 0

                rank[t, s1, k] = rankDict[state]
                
    
    # for END transition prob calculation
    h = []
    rankDict = dict()
    for i in range(num_nodes):
        curr_emission = emission(nodes[i], obs[-1])
        curr_transition = transition(nodes[i], "END")
        prob = scores[num_words, i, k] + np.log(curr_transition) + np.log(curr_emission)
        heapq.heappush(h, (prob, i))
    h_sorted = [heapq.heappop(h) for i in range(len(h))]
    h_sorted.reverse()
    for k in range(0, num_k):
        scores[num_words,i , k] = h_sorted[k][0]
        phi[num_words, i, k] = h_sorted[k][1]
        state = h_sorted[k][1]

        if state in rankDict:
            rankDict[state] = rankDict[state] + 1
        else:
            rankDict[state] = 0

        rank[num_words, i, k] = rankDict[state]
        
    # Put all the last items on the stack
    h = []

    # Get all the num_k from all the states
    for s1 in range(num_nodes):
        for k in range(num_k):
            prob = scores[num_words, s1, k]

            #Sort by the probability, but retain what state it came from and the k
            heapq.heappush(h, (prob, s1, k))

    # Then get sorted by the probability including its state and num_k
    h_sorted = [heapq.heappop(h) for i in range(len(h))]
    h_sorted.reverse()

    # init blank path
    path = np.zeros((num_k, num_words), int)
    path_probs = np.zeros((num_k, num_words),float)

    # Backpropagation
    for k in range(num_k):
        #The maximum probability and the state it came from
        max_prob = h_sorted[k][0]
        state = h_sorted[k][1]
        rankK = h_sorted[k][2]

        # Assign to output arrays
        path_probs[k][-1] = max_prob
        path[k][-1] = state

        #Then from t down to 0 store the correct sequence for t+1
        for t in range(num_words - 2, -1, -1):
            #The next state and its rank
            nextState = path[k][t+1]

            #Get the new state
            p = phi[t+1][nextState][rankK]

            #Pop into output array
            path[k][t] = p

            #Get the correct ranking for the next phi
            rankK = rank[t + 1][nextState][rankK]

    return path, path_probs, scores, phi

In [16]:
def preprocess_training_blank_row(data):
    start = time.process_time()   
    
    df= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False,engine="python",skip_blank_lines=False)
    # dropping null value columns to avoid errors 
    
    # new data frame with split value columns 
    df["x"], df["y"] = split_into_columns(df["original"])
    return df

In [17]:
def sentenceList(data):
    lines=[]
    line=[]
    x= data
    for label in x['x']:
        if pd.isnull(label)==False:
            line.append(label)
        else:
            lines.append(line)
            line = []
    return lines

In [18]:
def finalresult(sequence_log,predata_blank):
    dataframe = []
    count=0
    for i in range(len(sequence_log)):
        for text in sequence_log[i]:
            dataframe.append(text)
            count+=1
        dataframe.append("")
    dftest=pd.DataFrame(dataframe)
    final = pd.DataFrame()
    final['result'] = predata_blank['x'] + " " +dftest[0]
    return final

In [19]:
from csv import QUOTE_NONE
import time

k = 3
replaceWord = "#UNK#"
data_folders = ["CN","AL", "SG", "EN"]
for x in data_folders:
    print("Performing sentiment analysis for data folder ", x)
    train_data = "./data/{}/train".format(x)
    test_data = "./data/{}/dev.in".format(x)
    
    """## part2 ##"""
    train_arr = pd.read_csv(train_data, sep='\r\\n', index_col=False, engine="python", encoding="UTF-8").to_numpy()
    emission_dict, valid_x, xy_dict = emissionPara(train_arr, k, replaceWord)

    testdf = pd.read_csv(test_data, sep='\r\n', names=['x'],index_col=False,skip_blank_lines=False, engine="python", encoding="UTF-8")
    testdf = predict(testdf, xy_dict, replaceWord)
    print(testdf.head(3))
    
    print("Writing the final result to dev.p2.out...")
    testdf.to_csv('./output/{}/dev.p2.out'.format(x), header=False, index=False, na_rep="", sep="\n", quoting=QUOTE_NONE)
    print("Finished writing")
    
    """## part 3 ##"""
    train_df = pd.read_csv(train_data, sep='/r/n' ,index_col=False, engine="python", skip_blank_lines=False, encoding="UTF-8").to_numpy()
    transition_dic, subseq_count, y_count = transitionPara(train_df)
    nodes = list(y_count.keys())
    testdf_unprocess = pd.read_csv(test_data, sep='/n', delimiter=None, names=['x'],index_col=False,skip_blank_lines=False, engine="python")
    lines= sentenceList(testdf_unprocess)
    
    
    log_array =[]
    sequence_log=[]
    
    print("Performing Viterbi")
    start = time.time()
    for i in range(len(lines)):
        viterbioutput=viterbi(lines[i])
        log_array.append(viterbioutput)
        path = viterbi_backtrack(viterbioutput)
        sequence_log.append(path)
    end = time.time()
    
    print("Time taken for Viterbi and Backtrack", end - start)
    
    result = finalresult(sequence_log,testdf_unprocess)
    
    print("Writing the final result to dev.p3.out...")
    f = open('./output/{}/dev.p3.out'.format(x) ,'w')
    for word in result['result']:
        if pd.isnull(word) == False:
            f.write(word + '\n')
        else:
            f.write("" +"\n")
    f.close()
    print("Finished Writing.")


    """## part 4 ##"""
    print("Performing Viterbi for k best paths")
    sequence_log=[]
       
    start = time.time()
    k = 7
    for i in range(len(lines)):
        log_array.append(viterbioutput)
        path, path_probs, scores, phi = kViterbiParallel(nodes, lines[i], k)
        best_path_indexes = path[0]
        best_path = [nodes[x] for x in best_path_indexes]
        sequence_log.append(best_path)
        
    end = time.time()
    
    print("Time taken for K Best Parallel Viterbi", end - start)
    
    result = finalresult(sequence_log,testdf_unprocess)
    
    print("Writing the final result to dev.p4.out...")
    f = open('./output/{}/dev.p4.out'.format(x) ,'w')
    for word in result['result']:
        if pd.isnull(word) == False:
            f.write(word + '\n')
        else:
            f.write("" +"\n")
    f.close()
    print("Finished Writing.")

Performing sentiment analysis for data folder  CN
There are 168751 observations, 16402 values of x is to be replaced 
Predicting labels
0        一 I-negative
1    #UNK# B-negative
2    #UNK# B-negative
Name: x, dtype: object
Writing the final result to dev.p2.out...
Finished writing
Performing Viterbi


/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in log


Time taken for Viterbi and Backtrack 11.751538038253784
Writing the final result to dev.p3.out...
Finished Writing.
Performing Viterbi for k best paths


/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log
/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


Time taken for K Best Parallel Viterbi 106.15021395683289
Writing the final result to dev.p4.out...
Finished Writing.
Performing sentiment analysis for data folder  AL
There are 174948 observations, 3891 values of x is to be replaced 
Predicting labels
0    杭 B-CITY
1    州 I-CITY
2    市 I-CITY
Name: x, dtype: object
Writing the final result to dev.p2.out...
Finished writing
Performing Viterbi


/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/mnt/c/Users/estee/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in log


Time taken for Viterbi and Backtrack 196.39462113380432
Writing the final result to dev.p3.out...
Finished Writing.
Performing Viterbi for k best paths
Time taken for K Best Parallel Viterbi 3604.882967710495
Writing the final result to dev.p4.out...
Finished Writing.
Performing sentiment analysis for data folder  SG
There are 336220 observations, 47404 values of x is to be replaced 
Predicting labels
0       Tour B-positive
1    Scotland B-neutral
2           followers O
Name: x, dtype: object
Writing the final result to dev.p2.out...
Finished writing
Performing Viterbi
Time taken for Viterbi and Backtrack 7.978028774261475
Writing the final result to dev.p3.out...
Finished Writing.
Performing Viterbi for k best paths
Time taken for K Best Parallel Viterbi 82.87734913825989
Writing the final result to dev.p4.out...
Finished Writing.
Performing sentiment analysis for data folder  EN
There are 181627 observations, 15287 values of x is to be replaced 
Predicting labels
0     #UNK# B-UCP


Run the below script to get the evalscript results for part 2-4!

In [21]:
!python3 evalResult.py


 ----------- Evaluation Results for **AL**------------
*Part 2*

#Entity in gold data: 8408
#Entity in prediction: 19587

#Correct Entity : 1780
Entity  precision: 0.0909
Entity  recall: 0.2117
Entity  F: 0.1272

#Correct Sentiment : 1053
Sentiment  precision: 0.0538
Sentiment  recall: 0.1252
Sentiment  F: 0.0752

*Part3*

#Entity in gold data: 8408
#Entity in prediction: 8467

#Correct Entity : 3300
Entity  precision: 0.3897
Entity  recall: 0.3925
Entity  F: 0.3911

#Correct Sentiment : 2449
Sentiment  precision: 0.2892
Sentiment  recall: 0.2913
Sentiment  F: 0.2903

*Part4 - Only printing the evalScript results for best path*

#Entity in gold data: 8408
#Entity in prediction: 7294

#Correct Entity : 624
Entity  precision: 0.0855
Entity  recall: 0.0742
Entity  F: 0.0795

#Correct Sentiment : 51
Sentiment  precision: 0.0070
Sentiment  recall: 0.0061
Sentiment  F: 0.0065

 ----------- Evaluation Results for **EN**------------
*Part 2*

#Entity in gold data: 13179
#Entity in prediction: